<a href="https://colab.research.google.com/github/UrsachiGabriela/APAutomationSystem/blob/main/Introduction_to_CUDA_%2B_profiling_using_google_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to CUDA. Profiling CUDA application in Google Colab

Adapted from "An Even Easier Introduction to CUDA" ([_An Even Easier Introduction to CUDA_](https://developer.nvidia.com/blog/even-easier-introduction-cuda/)).



<img src="https://developer.download.nvidia.com/training/courses/T-AC-01-V1/CUDA_Cube_1K.jpeg" width="400">

## Starting Simple

We’ll start with a simple C++ program that adds the elements of two arrays with a million elements each.

In [ ]:
%%writefile add.cpp

#include <iostream>
#include <math.h>

// function to add the elements of two arrays
void add(int n, float *x, float *y, float *z)
{
  for (int i = 0; i < n; i++)
      z[i] = x[i] + y[i];
}

int main(void)
{
  int N = 1<<20; // 1M elements

  float *x = new float[N];
  float *y = new float[N];
  float *z = new float[N];

  // initialize x and y arrays on the host
  for (int i = 0; i < N; i++) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }

  // Run kernel on 1M elements on the CPU
  add(N, x, y, z);

  // Check for errors (all values should be 3.0f)
  float maxError = 0.0f;
  for (int i = 0; i < N; i++)
    maxError = fmax(maxError, fabs(z[i]-3.0f));
  std::cout << "Max error: " << maxError << std::endl;

  // Free memory
  delete [] x;
  delete [] y;
  delete [] z;

  return 0;
}

Writing add.cpp


Executing the above cell will save its contents to the file add.cpp.

The following cell will compile and run this C++ program.

In [ ]:
%%shell
g++ add.cpp -o add

Then run it:

In [ ]:
%%shell
./add

Max error: 0


As expected, it prints that there was no error in the summation and then exits. Now we want to get this computation running (in parallel) on the many cores of a GPU. It’s actually pretty easy to take the first steps.

First, we just have to turn our `add` function into a function that the GPU can run, called a *kernel* in CUDA. To do this, all we have to do is add the specifier `__global__` to the function, which tells the CUDA C++ compiler that this is a function that runs on the GPU and can be called from CPU code.

```cpp
// CUDA Kernel function to add the elements of two arrays on the GPU
__global__
void add(int n, float *dev_x, float *dev_y, float *dev_z)
{
  for (int i = 0; i < n; i++)
      dev_z[i] = dev_x[i] + dev_y[i];
}
```

These `__global__` functions are known as *kernels*, and code that runs on the GPU is often called *device code*, while code that runs on the CPU is *host code*.

## Memory Allocation and Data Transfer in CUDA

To compute on the GPU, we need to allocate memory accessible by the GPU.  To allocate data , call `cudaMalloc()`, which returns a pointer that you can access from device (GPU) code. To free the data, just pass the pointer to `cudaFree()`.

We just need to replace the calls to `new` in the code above with calls to `cudaMalloc`, and replace calls to `delete []` with calls to `cudaFree`.

```cpp
  // Allocate Memory -- accessible from GPU
  float *dev_x, *dev_y, *dev_z;
  cudaMalloc((void**)&dev_x, N*sizeof(float));
  cudaMalloc((void**)&dev_y, N*sizeof(float));
  cudaMalloc((void**)&dev_z, N*sizeof(float));
  ...

  // Free memory
  cudaFree(x);
  cudaFree(y);
  cudaFree(z);
```

To transfer data from CPU to GPU or vice versa, call `cudaMemcpy()` and specify the direction of the transfer:

*   `cudaMemcpyHostToDevice`- from CPU to GPU
*   `cudaMemcpyDeviceToHost` - from GPU to CPU
*   `...`




```cpp
  // Copy input vectors from host memory to GPU buffers.
  cudaMemcpy(dev_x, x, size * sizeof(int), cudaMemcpyHostToDevice);
  cudaMemcpy(dev_y, y, size * sizeof(int), cudaMemcpyHostToDevice);

  ...
```

Finally, we need to *launch* the `add()` kernel, which invokes it on the GPU. CUDA kernel launches are specified using the triple angle bracket syntax `<<< >>>`. We just have to add it to the call to `add` before the parameter list.

```cpp
add<<<1, 1>>>(N, dev_x, dev_y, dev_z);
```

We need the CPU to wait until the kernel is done before it accesses the results (because CUDA kernel launches don’t block the calling CPU thread). To do this we just call `cudaDeviceSynchronize()` before transferring the result back on CPU (`cudaMemcpy()`) and doing the final error checking.

Here’s the complete code:

In [ ]:
%%writefile add.cu

#include <iostream>
#include <math.h>
// Kernel function to add the elements of two arrays
__global__
void add(int n, float *dev_x, float *dev_y, float *dev_z)
{
  for (int i = 0; i < n; i++)
      dev_z[i] = dev_x[i] + dev_y[i];
}

int main(void)
{
  int N = 1<<20;
  float *x = new float[N];
  float *y = new float[N];
  float *z = new float[N];

  // Allocate Memory -- accessible from GPU
  float *dev_x, *dev_y, *dev_z;
  cudaMalloc((void**)&dev_x, N*sizeof(float));
  cudaMalloc((void**)&dev_y, N*sizeof(float));
  cudaMalloc((void**)&dev_z, N*sizeof(float));

  // initialize x and y arrays on the host
  for (int i = 0; i < N; i++) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }

  //transfer data to GPU
  cudaMemcpy(dev_x, x, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(dev_y, y, N*sizeof(float), cudaMemcpyHostToDevice);

  // Run kernel on 1M elements on the GPU
  add<<<1, 1>>>(N, dev_x, dev_y, dev_z);

  // Wait for GPU to finish before accessing on host
  cudaDeviceSynchronize();


  // transfer data back to host
  cudaMemcpy(z, dev_z, N*sizeof(float), cudaMemcpyDeviceToHost);

  // Check for errors (all values should be 3.0f)
  float maxError = 0.0f;
  for (int i = 0; i < N; i++)
    maxError = fmax(maxError, fabs(z[i]-3.0f));
  std::cout << "Max error: " << maxError << std::endl;

  // Free GPU and CPU memory
  cudaFree(dev_x);
  cudaFree(dev_y);
  cudaFree(dev_z);

  delete [] x;
  delete [] y;
  delete [] z;

  return 0;
}

Writing add.cu


In [ ]:
%%shell

nvcc add.cu -o add_cuda
./add_cuda

Max error: 0


This is only a first step, because as written, this kernel is only correct for a single thread, since every thread that runs it will perform the add on the whole array. Moreover, there is a [race condition](https://en.wikipedia.org/wiki/Race_condition) since multiple parallel threads would both read and write the same locations.

## Profile it!

The simplest way to find out how long the kernel takes to run is to run it with `nvprof`, the command line GPU profiler that comes with the CUDA Toolkit. Just type `nvprof ./add_cuda` on the command line:

In [ ]:
%%shell

nvprof  ./add_cuda

nvprof --print-gpu-trace ./add_cuda

ncu --set full -o add_cuda ./add_cuda #You can save the generate report and open it with Nsight Compute

==1322== NVPROF is profiling process 1322, command: ./add_cuda
Max error: 0
==1322== Profiling application: ./add_cuda
==1322== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   97.37%  123.25ms         1  123.25ms  123.25ms  123.25ms  add(int, float*, float*, float*)
                    1.34%  1.6937ms         1  1.6937ms  1.6937ms  1.6937ms  [CUDA memcpy DtoH]
                    1.30%  1.6403ms         2  820.14us  800.86us  839.42us  [CUDA memcpy HtoD]
      API calls:   59.12%  187.08ms         3  62.360ms  97.138us  186.86ms  cudaMalloc
                   38.95%  123.27ms         1  123.27ms  123.27ms  123.27ms  cudaDeviceSynchronize
                    1.64%  5.1783ms         3  1.7261ms  1.0182ms  3.1099ms  cudaMemcpy
                    0.18%  559.21us         3  186.40us  164.54us  199.44us  cudaFree
                    0.07%  223.80us         1  223.80us  223.80us  223.80us  cudaLaunchKernel
               

Let's make it faster with parallelism.

## Picking up the Threads

Now that you’ve run a kernel with one thread that does some computation, how do you make it parallel? The key is in CUDA’s `<<<1, 1>>>` syntax. This is called the execution configuration, and it tells the CUDA runtime how many parallel threads to use for the launch on the GPU. There are two parameters here, but let’s start by changing the second one: the number of threads in a thread block. CUDA GPUs run kernels using blocks of threads that are a multiple of 32 in size, so 256 threads is a reasonable size to choose.

```cpp
add<<<1, 256>>>(N, dev_x, dev_y, dev_Z);
```

If we run the code with only this change, it will do the computation once per thread, rather than spreading the computation across the parallel threads. To do it properly, we need to modify the kernel. CUDA C++ provides keywords that let kernels get the indices of the running threads. Specifically, `threadIdx.x` contains the index of the current thread within its block, and `blockDim.x` contains the number of threads in the block. We’ll just modify the loop to stride through the array with parallel threads.

```cpp
__global__
void add(int n, float *dev_x, float *dev_y, float *dev_z)
{
  int index = threadIdx.x;
  int stride = blockDim.x;
  for (int i = index; i < n; i += stride)
      dev_z[i] = dev_x[i] + dev_y[i];
}
```

The `add` function hasn’t changed that much. In fact, setting `index` to 0 and `stride` to 1 makes it semantically identical to the first version.

Here we save the file as add_block.cu and compile and run it in `nvprof` again.

In [ ]:
%%writefile add.cu

#include <iostream>
#include <math.h>
// Kernel function to add the elements of two arrays
__global__
void add(int n, float *dev_x, float *dev_y, float *dev_z)
{
  int index = threadIdx.x;
  int stride = blockDim.x;
  for (int i = index; i < n; i += stride)
      dev_z[i] = dev_x[i] + dev_y[i];
}

int main(void)
{
  int N = 1<<20;
  float *x = new float[N];
  float *y = new float[N];
  float *z = new float[N];

  // Allocate Memory -- accessible from GPU
  float *dev_x, *dev_y, *dev_z;
  cudaMalloc((void**)&dev_x, N*sizeof(float));
  cudaMalloc((void**)&dev_y, N*sizeof(float));
  cudaMalloc((void**)&dev_z, N*sizeof(float));

  // initialize x and y arrays on the host
  for (int i = 0; i < N; i++) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }

  //transfer data to GPU
  cudaMemcpy(dev_x, x, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(dev_y, y, N*sizeof(float), cudaMemcpyHostToDevice);

  // Run kernel on 1M elements on the GPU
  add<<<1, 256>>>(N, dev_x, dev_y, dev_z);

  // Wait for GPU to finish before accessing on host
  cudaDeviceSynchronize();


  // transfer data back to host
  cudaMemcpy(z, dev_z, N*sizeof(float), cudaMemcpyDeviceToHost);

  // Check for errors (all values should be 3.0f)
  float maxError = 0.0f;
  for (int i = 0; i < N; i++)
    maxError = fmax(maxError, fabs(z[i]-3.0f));
  std::cout << "Max error: " << maxError << std::endl;

  // Free GPU and CPU memory
  cudaFree(dev_x);
  cudaFree(dev_y);
  cudaFree(dev_z);

  delete [] x;
  delete [] y;
  delete [] z;

  return 0;
}

Overwriting add.cu


In [ ]:
%%shell
nvcc add.cu -o add_cuda

nvprof  ./add_cuda

nvprof --print-gpu-trace ./add_cuda

ncu -f --set full -o add_cuda ./add_cuda #You can save the generate report and open it with Nsight Compute

==2464== NVPROF is profiling process 2464, command: ./add_cuda
Max error: 0
==2464== Profiling application: ./add_cuda
==2464== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   43.13%  2.3993ms         1  2.3993ms  2.3993ms  2.3993ms  add(int, float*, float*, float*)
                   30.13%  1.6761ms         1  1.6761ms  1.6761ms  1.6761ms  [CUDA memcpy DtoH]
                   26.74%  1.4877ms         2  743.87us  728.41us  759.32us  [CUDA memcpy HtoD]
      API calls:   78.73%  194.00ms         3  64.665ms  70.035us  193.85ms  cudaMalloc
                   18.02%  44.399ms         1  44.399ms  44.399ms  44.399ms  cudaLaunchKernel
                    1.98%  4.8831ms         3  1.6277ms  880.78us  3.0374ms  cudaMemcpy
                    0.97%  2.4009ms         1  2.4009ms  2.4009ms  2.4009ms  cudaDeviceSynchronize
                    0.23%  576.68us         3  192.23us  181.99us  198.51us  cudaFree
               

That’s a big speedup (compare the time for the `add` kernel by looking at the `GPU activities` field), but not surprising since we went from 1 thread to 256 threads. Let’s keep going to get even more performance.

## Out of the Blocks

CUDA GPUs have many parallel processors grouped into Streaming Multiprocessors, or SMs. Each SM can run multiple concurrent thread blocks. As an example, a Tesla P100 GPU based on the [Pascal GPU Architecture](https://developer.nvidia.com/blog/inside-pascal/) has 56 SMs, each capable of supporting up to 2048 active threads. To take full advantage of all these threads, we should launch the kernel with multiple thread blocks.

By now you may have guessed that the first parameter of the execution configuration specifies the number of thread blocks. Together, the blocks of parallel threads make up what is known as the *grid*. Since we have `N` elements to process, and 256 threads per block, we just need to calculate the number of blocks to get at least `N` threads. We simply divide `N` by the block size (being careful to round up in case `N` is not a multiple of `blockSize`).

```cpp
int blockSize = 256;
int numBlocks = (N + blockSize - 1) / blockSize;
add<<<numBlocks, blockSize>>>(N, dev_x, dev_y, dev_z);
```

<img src="https://developer-blogs.nvidia.com/wp-content/uploads/2017/01/cuda_indexing.png" width="800">

We also need to update the kernel code to take into account the entire grid of thread blocks. CUDA provides `gridDim.x`, which contains the number of blocks in the grid, and `blockIdx.x`, which contains the index of the current thread block in the grid. Figure 1 illustrates the the approach to indexing into an array (one-dimensional) in CUDA using `blockDim.x`, `gridDim.x`, and `threadIdx.x`. The idea is that each thread gets its index by computing the offset to the beginning of its block (the block index times the block size: `blockIdx.x * blockDim.x`) and adding the thread’s index within the block (`threadIdx.x`). The code `blockIdx.x * blockDim.x + threadIdx.x` is idiomatic CUDA.

```cpp
__global__
void add(int n, float *dev_x, float *de_y, float *dev_z)
{
  int index = blockIdx.x * blockDim.x + threadIdx.x;
  int stride = blockDim.x * gridDim.x;
  for (int i = index; i < n; i += stride)
    dev_z[i] = dev_x[i] + dev_y[i];
}
```

The updated kernel also sets stride to the total number of threads in the grid (`blockDim.x * gridDim.x`). This type of loop in a CUDA kernel is often called a [*grid-stride*](https://developer.nvidia.com/blog/cuda-pro-tip-write-flexible-kernels-grid-stride-loops/) loop.

Save the file as `add_grid.cu` and compile and run it in `nvprof` again.

In [ ]:
%%writefile add.cu

#include <iostream>
#include <math.h>
// Kernel function to add the elements of two arrays
__global__
void add(int n, float *dev_x, float *dev_y, float *dev_z)
{
  int index = blockIdx.x * blockDim.x + threadIdx.x;
  int stride = blockDim.x * gridDim.x;
  for (int i = index; i < n; i += stride)
    dev_z[i] = dev_x[i] + dev_y[i];
}

int main(void)
{
  int N = 1<<20;
  float *x = new float[N];
  float *y = new float[N];
  float *z = new float[N];

  // Allocate Memory -- accessible from GPU
  float *dev_x, *dev_y, *dev_z;
  cudaMalloc((void**)&dev_x, N*sizeof(float));
  cudaMalloc((void**)&dev_y, N*sizeof(float));
  cudaMalloc((void**)&dev_z, N*sizeof(float));

  // initialize x and y arrays on the host
  for (int i = 0; i < N; i++) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }

  //transfer data to GPU
  cudaMemcpy(dev_x, x, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(dev_y, y, N*sizeof(float), cudaMemcpyHostToDevice);


  int blockSize = 256;
  int numBlocks = (N + blockSize - 1) / blockSize;
  add<<<numBlocks, blockSize>>>(N, dev_x, dev_y, dev_z);


  // Wait for GPU to finish before accessing on host
  cudaDeviceSynchronize();


  // transfer data back to host
  cudaMemcpy(z, dev_z, N*sizeof(float), cudaMemcpyDeviceToHost);

  // Check for errors (all values should be 3.0f)
  float maxError = 0.0f;
  for (int i = 0; i < N; i++)
    maxError = fmax(maxError, fabs(z[i]-3.0f));
  std::cout << "Max error: " << maxError << std::endl;

  // Free GPU and CPU memory
  cudaFree(dev_x);
  cudaFree(dev_y);
  cudaFree(dev_z);

  delete [] x;
  delete [] y;
  delete [] z;

  return 0;
}

Writing add.cu


In [ ]:
%%shell

nvcc add.cu -o add_cuda

nvprof  ./add_cuda

nvprof --print-gpu-trace ./add_cuda

ncu -f --set full -o add_cuda ./add_cuda #You can save the generate report and open it with Nsight Compute

==548== NVPROF is profiling process 548, command: ./add_cuda
Max error: 0
==548== Profiling application: ./add_cuda
==548== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   56.36%  1.8971ms         1  1.8971ms  1.8971ms  1.8971ms  [CUDA memcpy DtoH]
                   42.00%  1.4138ms         2  706.91us  700.94us  712.87us  [CUDA memcpy HtoD]
                    1.64%  55.231us         1  55.231us  55.231us  55.231us  add(int, float*, float*, float*)
      API calls:   49.32%  113.79ms         1  113.79ms  113.79ms  113.79ms  cudaLaunchKernel
                   48.08%  110.92ms         3  36.975ms  80.841us  110.73ms  cudaMalloc
                    2.24%  5.1565ms         3  1.7188ms  883.16us  3.3447ms  cudaMemcpy
                    0.27%  620.13us         3  206.71us  184.97us  221.51us  cudaFree
                    0.06%  135.22us       114  1.1860us     135ns  52.290us  cuDeviceGetAttribute
                    

# Generare graf

In [18]:
%%writefile graph_generator.cpp

#include <iostream>
#include <vector>
#include <fstream>
#include <cstdlib>
#include <ctime>
#include <algorithm>

using namespace std;

vector<vector<int>> generateRandomGraph(int numNodes, int numEdges) {
    vector<vector<int>> graph(numNodes);
    srand(time(0));

    for (int i = 0; i < numEdges; ++i) {
        int u = rand() % numNodes;
        int v = rand() % numNodes;

        // Evită buclele și muchiile duplicate
        while (u == v || find(graph[u].begin(), graph[u].end(), v) != graph[u].end()) {
            u = rand() % numNodes;
            v = rand() % numNodes;
        }
        graph[u].push_back(v);
        graph[v].push_back(u); // Dacă este un graf neorientat
    }

    return graph;
}

void writeGraphToFile(const string& filename, const vector<vector<int>>& graph) {
    ofstream outputFile(filename);
    if (!outputFile) {
        cerr << "Eroare la deschiderea fișierului pentru scriere!" << endl;
        return;
    }

    for (const auto& neighbors : graph) {
        for (size_t i = 0; i < neighbors.size(); ++i) {
            outputFile << neighbors[i];
            if (i != neighbors.size() - 1) {
                outputFile << " ";
            }
        }
        outputFile << endl; // Sfârșitul fiecărei liste de vecini
    }

    outputFile.close();
}

int main() {
    int numNodes = 1000; // De exemplu
    int numEdges = 5000;

    vector<vector<int>> graph = generateRandomGraph(numNodes, numEdges);

    string filename = "graph_output.txt";
    writeGraphToFile(filename, graph);

    cout << "Graf generat și salvat în " << filename << endl;

    return 0;
}



Overwriting graph_generator.cpp


In [19]:
!g++ graph_generator.cpp -o graph_generator
!./graph_generator


Graf generat și salvat în graph_output.txt


In [20]:
from google.colab import files
files.download('graph_output.txt')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# CPU

In [24]:
%%writefile bfs_cpu.cpp

#include <iostream>
#include <vector>
#include <queue>
#include <set>
#include <climits>
#include <fstream>
#include <sstream>
#include <string>

using namespace std;

vector<vector<int>> readGraphFromFile(const string& filename) {
    ifstream inputFile(filename);
    if (!inputFile) {
        cerr << "Eroare la deschiderea fisierului!" << endl;
        return {};
    }

    vector<vector<int>> graph;
    string line;

    while (getline(inputFile, line)) {
        // Parsare linie pentru vecini
        stringstream ss(line);
        vector<int> neighbors;
        int neighbor;

        while (ss >> neighbor) {
            neighbors.push_back(neighbor);
        }

        // Adăugare vecini în graful principal
        graph.push_back(neighbors);
    }

    inputFile.close();
    return graph;
}



void bfs_cpu(const vector<vector<int>>& adjacencyList, int start_node, vector<int>& distance) {
    queue<int> to_visit_queue;

    fill(distance.begin(), distance.end(), INT_MAX);
    distance[start_node] = 0;

    to_visit_queue.push(start_node);

    while (!to_visit_queue.empty()) {
        int current = to_visit_queue.front();
        to_visit_queue.pop();

        cout << current <<", distance: " << distance[current] << endl;

        for (int neighbor : adjacencyList[current]) {
            if (distance[neighbor] == INT_MAX) { // neighbor not visited yet
                distance[neighbor] = distance[current] + 1;

                to_visit_queue.push(neighbor);
            }
        }
    }
}

int main() {
    string filename = "graph_output.txt";
    vector<vector<int>> graph = readGraphFromFile(filename);

    //  for (int i = 0; i < graph.size(); ++i) {
    //     cout << "Node " << i << ": ";
    //     for (int neighbor : graph[i]) {
    //         cout << neighbor << " ";
    //     }
    //     cout << endl;
    // }

    // vector<vector<int>> graph = {
    //     {1, 2},
    //     {0, 3, 4},
    //     {0, 4, 5},
    //     {1, 6},
    //     {1, 2, 5},
    //     {2, 4},
    //     {3}
    // };

    int start_node = 0;
    vector<int> distance;
    distance = vector<int>(graph.size());

    bfs_cpu(graph, start_node, distance);

    return 0;
}

Overwriting bfs_cpu.cpp


In [25]:
%%shell
g++ bfs_cpu.cpp -o bfs_cpu

In [26]:
%%shell
./bfs_cpu

0, distance: 0
245, distance: 1
492, distance: 1
929, distance: 1
491, distance: 1
235, distance: 1
931, distance: 1
757, distance: 1
7, distance: 1
923, distance: 1
455, distance: 2
540, distance: 2
275, distance: 2
361, distance: 2
642, distance: 2
688, distance: 2
355, distance: 2
517, distance: 2
549, distance: 2
133, distance: 2
571, distance: 2
640, distance: 2
739, distance: 2
262, distance: 2
691, distance: 2
574, distance: 2
744, distance: 2
716, distance: 2
358, distance: 2
329, distance: 2
81, distance: 2
792, distance: 2
681, distance: 2
316, distance: 2
130, distance: 2
734, distance: 2
808, distance: 2
347, distance: 2
120, distance: 2
440, distance: 2
920, distance: 2
874, distance: 2
837, distance: 2
180, distance: 2
917, distance: 2
297, distance: 2
19, distance: 2
430, distance: 2
548, distance: 2
309, distance: 2
672, distance: 2
186, distance: 2
671, distance: 2
95, distance: 2
669, distance: 2
390, distance: 2
536, distance: 2
438, distance: 2
485, distance: 2
348,

# GPU

In [29]:
%%writefile bfs2.cu

#include <iostream>
#include <vector>
#include <climits>
#include <cuda_runtime.h>
#include <fstream>
#include <sstream>
#include <string>

using namespace std;


cudaError_t bfs_gpu(const vector<vector<int>> &graph, int start_node, vector<int> &distance);

__global__
void simpleKernel(int level) {
    printf("simple kernel\n");
}


__global__
void computeNextLevel(int *adjacencyList, int *offsets, int *distance, int *currentFrontier, int frontierSize, int *nextFrontier, int *nextFrontierSize, int level) {
    const int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid >= frontierSize) return;

    int currentNode = currentFrontier[tid];
    int start = offsets[currentNode];
    int end = offsets[currentNode + 1];

    for (int i = start; i < end; ++i) {
        int neighbor = adjacencyList[i];

        if (atomicCAS(&distance[neighbor], INT_MAX, level + 1) == INT_MAX) {
            int position = atomicAdd(nextFrontierSize, 1);
            nextFrontier[position] = neighbor;
        }
    }
}

cudaError_t bfs_gpu(const vector<vector<int>> &graph, int start_node, vector<int> &distance) {
    cudaError_t cudaStatus;

    int frontier_size = 1;
    const int NEXT_FRONTIER_SIZE = 0;
    int level = 0;
    int threads_per_block = 256;

    int num_nodes = graph.size();

    vector<int> adjacencyList;
    vector<int> offsets(num_nodes + 1, 0);

    for (int i = 0; i < num_nodes; ++i) {
        offsets[i + 1] = offsets[i] + graph[i].size();
        adjacencyList.insert(adjacencyList.end(), graph[i].begin(), graph[i].end());
    }

    int *d_adjacencyList = 0;
    int *d_offsets = 0;
    int *d_distance = 0;
    int *d_frontier = 0;
    int *d_nextFrontier = 0;
    int *d_nextFrontierSize = 0;

    cudaStatus = cudaMalloc((void**)&d_adjacencyList, adjacencyList.size() * sizeof(int));
    if (cudaStatus != cudaSuccess) {
        fprintf(stderr, "cudaMalloc failed for d_adjacencyList!");
        goto Error;
    }

    cudaStatus = cudaMalloc((void**)&d_offsets, offsets.size() * sizeof(int));
    if (cudaStatus != cudaSuccess) {
        fprintf(stderr, "cudaMalloc failed for d_offsets!");
        goto Error;
    }

    cudaStatus = cudaMalloc((void**)&d_distance, num_nodes * sizeof(int));
    if (cudaStatus != cudaSuccess) {
        fprintf(stderr, "cudaMalloc failed for d_distance!");
        goto Error;
    }

    cudaStatus = cudaMalloc((void**)&d_frontier, num_nodes * sizeof(int));
    if (cudaStatus != cudaSuccess) {
        fprintf(stderr, "cudaMalloc failed for d_frontier!");
        goto Error;
    }

    cudaStatus = cudaMalloc((void**)&d_nextFrontier, num_nodes * sizeof(int));
    if (cudaStatus != cudaSuccess) {
        fprintf(stderr, "cudaMalloc failed for d_nextFrontier!");
        goto Error;
    }

    cudaStatus = cudaMalloc((void**)&d_nextFrontierSize, sizeof(int));
    if (cudaStatus != cudaSuccess) {
        fprintf(stderr, "cudaMalloc failed for d_nextFrontierSize!");
        goto Error;
    }


    // host to device
    cudaMemcpy(d_adjacencyList, adjacencyList.data(), adjacencyList.size() * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_offsets, offsets.data(), offsets.size() * sizeof(int), cudaMemcpyHostToDevice);

    distance.assign(num_nodes, INT_MAX);
    distance[start_node] = 0;
    cudaMemcpy(d_distance, distance.data(), num_nodes * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_frontier, &start_node, sizeof(int), cudaMemcpyHostToDevice);

    while (frontier_size > 0) {
        cudaMemcpy(d_nextFrontierSize, &NEXT_FRONTIER_SIZE, sizeof(int), cudaMemcpyHostToDevice);

        int *d_currentQueue = (level % 2 == 0) ? d_frontier : d_nextFrontier;
        int *d_nextQueue = (level % 2 == 0) ? d_nextFrontier : d_frontier;

        int blocks = (frontier_size + threads_per_block - 1) / threads_per_block;
        computeNextLevel<<<blocks, threads_per_block>>>(d_adjacencyList, d_offsets, d_distance, d_currentQueue, frontier_size, d_nextQueue, d_nextFrontierSize, level);

        cudaError_t err = cudaGetLastError();
        if (err != cudaSuccess) {
            cout << "CUDA error: " << err << endl;
            exit(-1);
        }
        cudaDeviceSynchronize();

        cudaMemcpy(&frontier_size, d_nextFrontierSize, sizeof(int), cudaMemcpyDeviceToHost);

        //cout << "Level " << level << ", frontier size: " << frontier_size << endl;

        ++level;
    }

    // device to host
    cudaMemcpy(distance.data(), d_distance, num_nodes * sizeof(int), cudaMemcpyDeviceToHost);

Error:
    cudaFree(d_adjacencyList);
    cudaFree(d_offsets);
    cudaFree(d_distance);
    cudaFree(d_frontier);
    cudaFree(d_nextFrontier);
    cudaFree(d_nextFrontierSize);

    return cudaStatus;
}



vector<vector<int>> readGraphFromFile(const string& filename) {
    ifstream inputFile(filename);
    if (!inputFile) {
        cerr << "Eroare la deschiderea fisierului!" << endl;
        return {};
    }

    vector<vector<int>> graph;
    string line;

    while (getline(inputFile, line)) {
        // Parsare linie pentru vecini
        stringstream ss(line);
        vector<int> neighbors;
        int neighbor;

        while (ss >> neighbor) {
            neighbors.push_back(neighbor);
        }

        // Adăugare vecini în graful principal
        graph.push_back(neighbors);
    }

    inputFile.close();
    return graph;
}

int main() {
    string filename = "graph_output.txt";
    vector<vector<int>> graph = readGraphFromFile(filename);



    int start_node = 0;
    vector<int> distance;

    cudaError_t cudaStatus = bfs_gpu(graph, start_node, distance);

    if (cudaStatus != cudaSuccess) {
        fprintf(stderr, "bfs_gpu failed!");
        return 1;
    }

    for (int i = 0; i < distance.size(); ++i) {
        cout << "Node " << i << ", Distance: " << distance[i] << endl;
    }

    return 0;
}

Overwriting bfs2.cu


That's another big speedup from running multiple blocks!

In [30]:
%%shell

nvcc bfs2.cu -o bfs2

nvprof  ./bfs2

nvprof --print-gpu-trace ./bfs2

ncu -f --set full -o bfs2 ./bfs2

==5989== NVPROF is profiling process 5989, command: ./bfs2
Node 0, Distance: 0
Node 1, Distance: 3
Node 2, Distance: 4
Node 3, Distance: 4
Node 4, Distance: 4
Node 5, Distance: 3
Node 6, Distance: 4
Node 7, Distance: 1
Node 8, Distance: 4
Node 9, Distance: 3
Node 10, Distance: 4
Node 11, Distance: 3
Node 12, Distance: 3
Node 13, Distance: 4
Node 14, Distance: 4
Node 15, Distance: 3
Node 16, Distance: 3
Node 17, Distance: 3
Node 18, Distance: 3
Node 19, Distance: 2
Node 20, Distance: 3
Node 21, Distance: 3
Node 22, Distance: 4
Node 23, Distance: 4
Node 24, Distance: 4
Node 25, Distance: 4
Node 26, Distance: 4
Node 27, Distance: 4
Node 28, Distance: 3
Node 29, Distance: 3
Node 30, Distance: 3
Node 31, Distance: 3
Node 32, Distance: 3
Node 33, Distance: 3
Node 34, Distance: 4
Node 35, Distance: 4
Node 36, Distance: 3
Node 37, Distance: 3
Node 38, Distance: 3
Node 39, Distance: 3
Node 40, Distance: 3
Node 41, Distance: 3
Node 42, Distance: 3
Node 43, Distance: 4
Node 44, Distance: 4
Node 4